In [ ]:
import os
import json
import numpy as np
from PIL import Image
from pycocotools import mask as coco_mask

def encode_mask_as_rle(mask_array):
    """
    Encodes a binary mask as COCO RLE format.
    :param mask_array: NumPy array of shape (height, width).
    :return: COCO RLE encoded segmentation.
    """
    rle = coco_mask.encode(np.asfortranarray(mask_array.astype(np.uint8)))
    rle['counts'] = rle['counts'].decode('utf-8')  # Convert from bytes to string for JSON compatibility
    return rle

def convert_png_to_coco_rle(image_path, image_id, category_id, starting_annotation_id=1):
    """
    Converts a PNG mask into COCO RLE format with one annotation per instance.
    :param image_path: Path to the PNG mask file.
    :param image_id: Unique ID for the image.
    :param category_id: ID for the object category.
    :param starting_annotation_id: Annotation id counter starting value.
    :return: COCO-style JSON dictionary.
    """
    # Load and process the mask
    mask = np.array(Image.open(image_path).convert("I"))
    height, width = mask.shape
    # Get all unique instance labels (excluding background label 0)
    instance_ids = np.unique(mask)
    instance_ids = instance_ids[instance_ids != 0]
    
    annotations = []
    annotation_id = starting_annotation_id

    for inst_id in instance_ids:
        # Create a binary mask for this instance
        instance_mask = (mask == inst_id).astype(np.uint8)
        pos = np.where(instance_mask > 0)
        if pos[0].size == 0 or pos[1].size == 0:
            continue  # skip empty instances
        
        # Compute bounding box in the format [x, y, width, height]
        x_min = int(np.min(pos[1]))
        y_min = int(np.min(pos[0]))
        x_max = int(np.max(pos[1]))
        y_max = int(np.max(pos[0]))
        bbox = [x_min, y_min, x_max - x_min, y_max - y_min]
        area = int(np.sum(instance_mask))
        
        # Encode the instance mask in RLE format
        rle_segmentation = encode_mask_as_rle(instance_mask)
        
        annotation = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,
            "segmentation": rle_segmentation,
            "area": area,
            "bbox": bbox,
            "iscrowd": 0
        }
        annotations.append(annotation)
        annotation_id += 1

    coco_json = {
        "info": {
            "description": "Converted PNG mask to COCO RLE",
            "url": "n/a",
            "version": "n/a",
            "year": 2025,
            "contributor": "Automated Script",
            "date_created": "2025/05/24"
        },
        "licenses": [{"url": "n/a", "id": 0, "name": "placeholder license"}],
        "images": [{
            "license": 0,
            "file_name": os.path.basename(image_path),
            "coco_url": "n/a",
            "height": height,
            "width": width,
            "date_captured": "",
            "id": image_id,
            "tag_ids": []
        }],
        "annotations": annotations
    }
    
    return coco_json

def fix_filename(input_filename):
    """
    If the input filename does not include an underscore, insert an underscore.
    """
    base, ext = os.path.splitext(input_filename)
    if base.startswith("TN") and not base.startswith("TN_"):
        base = "TN_" + base[len("TN"):]
    return base + ext

def extract_index(filename):
    base = os.path.splitext(filename)[0]  # Remove extension
    if base.startswith("TN_"):
        try:
            return int(base.split("_")[1])  # Extract the numeric part
        except ValueError:
            return -1  # Fallback if parsing fails
    elif base.startswith("TN"):
        try:
            return int(base[2:])  # Handle case like "TN0"
        except ValueError:
            return -1
    return -1  # Fallback for unexpected formats


def process_folder(image_folder, output_folder):
    """
    Loops through PNG masks in a folder and creates separate JSON files for each (without .png extension)
    using adjusted naming conventions.
    :param image_folder: Folder containing PNG mask images.
    :param output_folder: Folder to save JSON files.
    """
    os.makedirs(output_folder, exist_ok=True)
    #image_id = 0  # Unique ID counter for images
    annotation_id = 1  # Unique ID counter for annotations

    #png_files = sorted([f for f in os.listdir(image_folder) if f.endswith(".png")])
    png_files = sorted(
    [f for f in os.listdir(image_folder) if f.endswith(".png")],
    key=extract_index)


    for filename in png_files:
        image_id = extract_index(filename)
        image_path = os.path.join(image_folder, filename)
        json_filename = os.path.splitext(filename)[0] + ".json"
        #json_filename = f"TN_{image_id}.json"
        json_path = os.path.join(output_folder, json_filename)

        coco_json = convert_png_to_coco_rle(
            image_path,
            image_id=image_id,
            category_id=1,
            starting_annotation_id=annotation_id
        )

        if coco_json and "annotations" in coco_json:
            annotation_id += len(coco_json["annotations"])

        if coco_json:
            with open(json_path, "w") as f:
                json.dump(coco_json, f, indent=4)
            print(f"Saved: {json_path}")

        #image_id += 1

if __name__ == "__main__":
    # Specify the input folder (where PNG masks are stored)
    input_folder = "insert_path"
    # Specify the output folder (where JSON files will be saved)
    output_folder = "insert_path"
    process_folder(input_folder, output_folder)